In [1]:
# HIDDEN - sets path

import sys
sys.path.insert(1, '../..')

# work with times

## Client
It is important to set the timezone when creating the client. All output times will be given in this timezone, and timezone-unaware inputs are assumed to be in this timezone.

In [2]:
import keyring
from trendminer_interface import TrendMinerClient

url = "https://cs.trendminer.net"
client_id = "tmInterfaceClient"

client = TrendMinerClient(
    url = url,
    client_id = client_id,
    client_secret=keyring.get_password(url, client_id),
    tz="Europe/Brussels"
)

## Timstamps
All single timestamps are returned as `datetime.datetime` instances. Timestamp string inputs will be converted automaticallly. For string inputs, the date portion is assumed to be in YYYY-MM-DD.

In [3]:
# Current time
client.time.now()

datetime.datetime(2023, 1, 11, 12, 29, 33, 109118, tzinfo=<DstTzInfo 'Europe/Brussels' CET+1:00:00 STD>)

In [4]:
# Explicitly convert string to datetime
client.time.datetime("2021-01-12 14:05:00.000")

datetime.datetime(2021, 1, 12, 14, 5, tzinfo=<DstTzInfo 'Europe/Brussels' CET+1:00:00 STD>)

## Durations
All durations are returned as `datetime.timedelta`. Input strings and numeric values are coverted automatically.

In [5]:
# Strings are interpreted like they are in the TrendMiner UX
client.time.timedelta("4d 12h 5m 3s")

datetime.timedelta(days=4, seconds=43503)

In [6]:
# Numeric inputs are assumed to represent the number of seconds
client.time.timedelta(1800)

datetime.timedelta(seconds=1800)

## Intervals
Intervals are described by a fixed start and end time. They are a core element in the package, and in TrendMiner in general. TrendHub views, context items, search periods, ... they are all defined by an interval.

In [7]:
# Intervals can be made from two timestamps (datetimes or strings)
client.time.interval("2021-01-01", "2022-01-01")

<< Interval | 2021-01-01 00:00:00+01:00 | 365 days, 0:00:00 >>

In [8]:
# For as single input, the end is the current time
client.time.interval("2023-01-10 12:00")

<< Interval | 2023-01-10 12:00:00+01:00 | 1 day, 0:29:33.155636 >>

In [9]:
# Duration-like string inputs als take the current time as the end
client.time.interval("5d 12h")

<< Interval | 2023-01-06 00:29:33.163306+01:00 | 5 days, 12:00:00 >>

### Properties and methods
Intervals have a set of useful properties and methods implemented

In [10]:
# Basic properties
interval = client.time.interval("9h")

print(f"Start: {interval.start}")
print(f"End: {interval.end}")
print(f"Duration: {interval.duration}")

Start: 2023-01-11 03:29:33.181966+01:00
End: 2023-01-11 12:29:33.181966+01:00
Duration: 9:00:00


In [11]:
# Shifting an interval
interval = interval.shift("-1h")

print(f"Start: {interval.start}")
print(f"End: {interval.end}")
print(f"Duration: {interval.duration}")

Start: 2023-01-11 02:29:33.181966+01:00
End: 2023-01-11 11:29:33.181966+01:00
Duration: 9:00:00


In [12]:
# Rounding an interval down to a given resolution
interval = interval.narrow("1h")
                           
print(f"Start: {interval.start}")
print(f"End: {interval.end}")
print(f"Duration: {interval.duration}")

Start: 2023-01-11 03:00:00+01:00
End: 2023-01-11 11:00:00+01:00
Duration: 8:00:00


In [13]:
# Splitting an interval based on a maximal duration
interval.split("2h")

[<< Interval | 2023-01-11 03:00:00+01:00 | 2:00:00 >>,
 << Interval | 2023-01-11 05:00:00+01:00 | 2:00:00 >>,
 << Interval | 2023-01-11 07:00:00+01:00 | 2:00:00 >>,
 << Interval | 2023-01-11 09:00:00+01:00 | 2:00:00 >>]

In [14]:
# Taking random samples of a given length from a longer interval
interval.sample(
    duration="15m", # resulting interval duration
    n=4, # number of intervals
    overlap=False, # whether the intervals are allowed to overlap
    resolution="1m", # rounded to given resolution
)

[<< Interval | 2023-01-11 04:36:00+01:00 | 0:15:00 >>,
 << Interval | 2023-01-11 05:21:00+01:00 | 0:15:00 >>,
 << Interval | 2023-01-11 05:32:00+01:00 | 0:15:00 >>,
 << Interval | 2023-01-11 09:13:00+01:00 | 0:15:00 >>]

## Periods
Periods are basically live intervals representing the last x time. Their start and end times are constantly moving, which makes them hard to work with. Typically, you want to be working with intervals, not periods. 

Periods are occasionally returned from the appliance when something is represented by a live, moving timeframe.

In [15]:
# A period can be defined from a duration
period = client.time.period("1d")
period

<< Period | 1 day, 0:00:00 >>

In [16]:
# A period is always moving
print(period.start)
print(period.start)

2023-01-10 12:29:33.333207+01:00
2023-01-10 12:29:33.333787+01:00


In [17]:
# We can freeze a period into a fixed interval
period.freeze()

<< Interval | 2023-01-10 12:29:33.339648+01:00 | 1 day, 0:00:00 >>

In [18]:
# Period methods will implicitly freeze the period
period.split("8h")

[<< Interval | 2023-01-10 12:29:33.348321+01:00 | 8:00:00 >>,
 << Interval | 2023-01-10 20:29:33.348321+01:00 | 8:00:00 >>,
 << Interval | 2023-01-11 04:29:33.348321+01:00 | 8:00:00 >>]